In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
  .remote("sc://192.168.1.7:15002") \
  .appName("UDFTransformation") \
  .config("spark.sql.ansi.enabled", "false") \
  .config("spark.sql.execution.pythonUDF.arrow.enabled", "true") \
  .getOrCreate()

# limit() shows a nice HTML table in Jupyter, while show() prints plain text  
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

spark

In [2]:
from pyspark.sql.functions import udf

@udf(returnType='int')
def mult_by_3(s: int) -> int:
  return s * 3

df = spark.createDataFrame([(4, ), (5, ), (6, )], ['value'])
df

value
4
5
6


In [3]:
dff = df.withColumn('value_x3', mult_by_3(df.value))
dff

value,value_x3
4,12
5,15
6,18


In [4]:
import pandas as pd
import pyspark.pandas as ps
from pyspark.sql.functions import pandas_udf

@pandas_udf("int")
def sub_2(s: pd.Series) -> pd.Series:
  return s - 2

dffs = dff.withColumn('value_minus_2', sub_2(dff.value))
dffs

c:\Users\plancha\spark-lab1\.venv\lib\site-packages\pyspark\pandas\__init__.py:43: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


value,value_x3,value_minus_2
4,12,2
5,15,3
6,18,4


In [ ]:
spark.stop()